In [773]:
import json
import pandas as pd
import warnings
import seaborn as sns
from pyprojroot import here
import plotly.express as px
import numpy as np
import math

methods = ["JustCopy", "TimeGAN", "Time-Transformer", "TransFusion", "TTS-GAN", "TimeVQVAE"]
datasets = ["D2", "D3", "D4", "D5", "D6", "D7"]

rows = []

for method in methods:
    for dataset in datasets:
        file_candidates = list(here('result').glob(f'numeric_{method}_{dataset}_*.json'))
        if len(file_candidates) != 1:
            warnings.warn(f"Ignoring {method} {dataset}: Expected one result file for {method} {dataset}, instead matched {file_candidates}.")
            continue
        file_path = file_candidates[0]
        
        with open(file_path) as f:
            data = json.load(f)
        
        rows.append(((method, dataset), data))

df_all = pd.DataFrame.from_dict(dict(rows), orient="index")
df_all.index = pd.MultiIndex.from_tuples(df_all.index, names=["Method", "Dataset"])

# load timings
def timings_path(method):
    paths = list(here("models").glob(f"*{method}*/timings.csv"))
    assert len(paths) == 1
    return paths[0]

timings = pd.DataFrame()
for method in methods:
    df = pd.read_csv(timings_path(method))
    df["Method"] = method
    timings = pd.concat([timings, df], ignore_index=True)

_timings = timings.melt(id_vars="Method", var_name="Dataset", value_name="Time").set_index(["Method", "Dataset"]).sort_index(level="Method")

measures_order = ["PS", "DS", "C-FID","MDD", "ACD",	"SD", "KD", "ED", "DTW", "Time"]
ranking_order = ["JustCopy", "TransFusion", "TimeVQVAE", "Time-Transformer", "TimeGAN", "TTS-GAN"]

def sortdf(df):
    return df.sort_index(level=0, key=lambda x: pd.Categorical(x, categories=ranking_order, ordered=True))

df_all = pd.concat([df_all, _timings], axis = 1).loc[ranking_order][measures_order]

In [774]:
cm = sns.diverging_palette(h_neg=130, h_pos=0, as_cmap=True)


def style_results_table(df, separate_models=False):
    sci_columns = {"ED", "DTW", "C-FID"}
    decimal_columns = set([col[0] if isinstance(col, tuple) else col for col in df.columns]) - sci_columns

    styled = df.style

    # === Formatting === #

    import math
    import pandas as pd

    def decimal_fmt(x, sig=3):
        if pd.isna(x):
            return ""
        if x == 0:
            return "0"
        digits = sig - int(math.floor(math.log10(abs(x)))) - 1
        rounded = round(x, max(digits, 0))
        s = f"{rounded:.{max(digits,0)}f}".rstrip("0").rstrip(".")
        return s

    def sci_fmt_unicode(x):
        if pd.isna(x):
            return ""
        if x == 0:
            return "0"
        exp = math.floor(math.log10(abs(x)))
        coeff = x / 10**exp
        if exp in {0,1,-1}:
            return f"{coeff:.3g}"
        superscripts = str.maketrans("0123456789-", "⁰¹²³⁴⁵⁶⁷⁸⁹⁻")
        return f"{coeff:.3g}×10{str(exp).translate(superscripts)}"

    fmt = {}

    for col in df.columns:
        colname = col[0] if isinstance(col, tuple) else col
        if colname in sci_columns:
            fmt[col] = sci_fmt_unicode
        elif colname in decimal_columns:
            fmt[col] = decimal_fmt

    styled = styled.format(fmt)

    # === Background Gradients === #

    ranges = {
        "DS": (0.012, 0.476),
        "PS": (0.113, 0.279),
        "C-FID": (0, 1),
        "MDD": (0.0231, 1.05),
        "ACD": (0, 1),
        "SD": (0, 1),
        "KD": (0, 2.16),
        "ED": (0, 4),
        "DTW": (0, 14),
        "Time": (0, 961),
    }

    for col in df.columns:
        colname = col[0] if isinstance(col, tuple) else col
        if colname in ranges:
            vmin, vmax = ranges[colname]
            styled = styled.background_gradient(
                cmap=cm,
                vmin=vmin,
                vmax=vmax,
                subset=[col]   # important: now accepts MultiIndex column keys
            )


    if separate_models:
        level0 = df.index.get_level_values(0)
        boundaries = np.where(level0[:-1] != level0[1:])[0]

        def highlight_boundaries(row):
            i = df.index.get_loc(row.name)  # positional index
            styles = [''] * row.size
            if (i - 1) in boundaries:
                styles = ['border-top: 2px solid white'] * row.size
            return styles

        # --- apply to styler object ---
        styled = styled.apply(highlight_boundaries, axis=1)

    return styled



def group_measures(df):
    category_map = {
    "Utility": ["PS"],
    "Fidelity": ["DS", "C-FID", "MDD", "ACD", "SD", "KD", "ED", "DTW"],
    "Training Efficiency": ["Time"]
    }

    # Create a list of tuples (top-level, column name)
    new_cols = []
    for cat, cols in category_map.items():
        for col in cols:
            new_cols.append((cat, col))

    # Reorder df columns to match the new structure
    df = df[[col for _, col in new_cols]]

    # Apply MultiIndex to columns
    df.columns = pd.MultiIndex.from_tuples(new_cols)
    return df

df_no_copy = df_all.drop(index="JustCopy", level="Method")
tab_full_results = style_results_table(df_all, separate_models=True)
tab_full_results

In [775]:
with open("full_results_table.tex", "w") as f:
    f.write(tab_full_results.to_latex())

In [776]:
def merge_datasets_mean(df, datasets_to_merge=[], new_name="D2-D5"):
    # Select rows to merge
    to_merge = df.loc[pd.IndexSlice[:, datasets_to_merge], :]
    
    # Compute mean grouped by Method
    merged = to_merge.groupby(level=0).mean()
    merged.index = pd.MultiIndex.from_product([merged.index, [new_name]], names=df.index.names)
    
    # Drop original 
    df_dropped = df.drop(datasets_to_merge, level=1)
    
    # Concatenate
    df_new = pd.concat([df_dropped, merged]).sort_index(level=0, key=lambda x: pd.Categorical(x, categories=ranking_order, ordered=True))

    df_new.index = df_new.index.rename(['Method','Seq. Length'])
    
    return df_new

def long_vs_short(df):
    return merge_datasets_mean(merge_datasets_mean(df, ["D2","D5"], "short (24)"), ["D3", "D6"], "long (128)").drop(level=1, index="D4").drop(level=1, index="D7")

df_long_vs_short = long_vs_short(df_all)
tab_long_vs_short = style_results_table(df_long_vs_short, separate_models=True)
tab_long_vs_short

In [777]:
def long_or_short(df):
    df = long_vs_short(df)
    out = pd.DataFrame(index=df.index.levels[0], columns=df.columns)

    for lvl1 in df.index.levels[0]:
        subset = df.xs(lvl1, level=0)
        min_idx = subset.idxmin()
        out.loc[lvl1] = min_idx


    def color_cells(value):
        if value == "long (128)":
            return 'background-color: lightseagreen'
        if value == "short (24)":
            return 'background-color: lightblue'

    return out.style.applymap(color_cells)

tab_long_or_short = long_or_short(df_all.drop(index="JustCopy").drop(index="TTS-GAN"))
tab_long_or_short

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4150: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Method,,,,,,,,,,
Time-Transformer,long (128),long (128),short (24),short (24),short (24),short (24),short (24),short (24),short (24),short (24)
TimeGAN,short (24),short (24),short (24),short (24),short (24),short (24),long (128),short (24),short (24),short (24)
TimeVQVAE,long (128),long (128),long (128),long (128),short (24),long (128),long (128),short (24),short (24),short (24)
TransFusion,long (128),short (24),long (128),long (128),short (24),long (128),long (128),short (24),short (24),short (24)


In [778]:
def long_minus_short(df):
    out = long_vs_short(df)

    out = out.groupby(level=0).diff()

    return out.loc[pd.IndexSlice[:, 'short (24)'], :]

style_results_table(
    long_minus_short(
        df_all
            .drop(index="JustCopy", level=0)
            .drop(index="TTS-GAN", level=0)
        ))

,,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Method,Seq. Length,,,,,,,,,,
TransFusion,short (24),0.000997,-0.0468,5.2×10⁻²,0.207,-0.0636,0.218,1.05,-1.57,-6.22,-682
TimeVQVAE,short (24),0.0305,0.00416,6.65×10⁻⁴,0.022,-0.198,0.0411,0.143,-1.28,-5.27,-17.5
Time-Transformer,short (24),0.0437,0.0133,-1.4,-0.172,-0.339,-0.0808,-0.341,-1.34,-5.71,-7
TimeGAN,short (24),-0.0133,-0.0747,-2.63,-0.391,-0.925,-0.042,0.0546,-1.51,-5.93,-1031


In [779]:
style_results_table(df_all.groupby(level=0).agg(["min","max", "mean"]).loc[ranking_order])

In [780]:
style_results_table(df_all.groupby(level=0).quantile([.25,.5,.75]).unstack(level=1).loc[ranking_order])

In [781]:
style_results_table(
    df_all
        .drop(index="TTS-GAN", level="Method")
        .drop(index="JustCopy", level="Method")
        .groupby(level="Dataset").agg(["min", "max", "mean"])
        .sort_values(("DS","mean"))
    )

In [782]:
style_results_table(
    df_all
        .groupby(level="Method").mean()
        .loc[ranking_order]
    )

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Method,,,,,,,,,,
JustCopy,0.113,0.012,-6.73×10⁻¹³,0.0231,0,0,0,0,0,0
TransFusion,0.114,0.115,2.41×10⁻²,0.344,0.122,0.176,0.718,2.1,8.13,527
TimeVQVAE,0.146,0.379,2.3,0.509,0.551,0.315,1.27,1.99,7.73,84.8
Time-Transformer,0.143,0.396,2.11,0.534,0.495,0.378,1.41,2.03,7.91,42.5
TimeGAN,0.143,0.286,2.49,0.583,0.717,0.346,1.31,2.14,8.22,961
TTS-GAN,0.279,0.476,5.25×10¹⁶,1.05,6.71,1,2.16,4.68×10⁷,1.52×10⁸,652


In [783]:
style_results_table(df_all.groupby("Method").mean()["PS"].to_frame().sort_values("PS"))

,PS
Method,
JustCopy,0.113
TransFusion,0.114
Time-Transformer,0.143
TimeGAN,0.143
TimeVQVAE,0.146
TTS-GAN,0.279


In [784]:
style_results_table(df_all.groupby("Method").mean()["Time"].to_frame().sort_values("Time").sort_values("Time"))

,Time
Method,
JustCopy,0
Time-Transformer,42.5
TimeVQVAE,84.8
TransFusion,527
TTS-GAN,652
TimeGAN,961


In [785]:
style_results_table(
    df_all
        .groupby("Method").mean()
        .drop(columns=["Time", "PS"])
        .loc[ranking_order]
        )

,DS,C-FID,MDD,ACD,SD,KD,ED,DTW
Method,,,,,,,,
JustCopy,0.012,-6.73×10⁻¹³,0.0231,0,0,0,0,0
TransFusion,0.115,2.41×10⁻²,0.344,0.122,0.176,0.718,2.1,8.13
TimeVQVAE,0.379,2.3,0.509,0.551,0.315,1.27,1.99,7.73
Time-Transformer,0.396,2.11,0.534,0.495,0.378,1.41,2.03,7.91
TimeGAN,0.286,2.49,0.583,0.717,0.346,1.31,2.14,8.22
TTS-GAN,0.476,5.25×10¹⁶,1.05,6.71,1,2.16,4.68×10⁷,1.52×10⁸


## Spider Plots

In [786]:
rankings = df_no_copy.unstack(level=1).rank()

ranking_by_metric = rankings.stack().reset_index().groupby("Method").mean()
spider_by_metric = ranking_by_metric.reset_index().melt(id_vars="Method")
fig_spider_by_metric = px.line_polar(spider_by_metric, r="value", theta="variable", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [5.9, 1], "dtick": 1}})
fig_spider_by_metric

In [787]:
ranking_by_dataset = rankings.stack(level=0).reset_index().groupby("Method").mean()
spider_by_dataset = ranking_by_dataset.reset_index().melt(id_vars="Method")
fig_spider_by_dataset = px.line_polar(spider_by_dataset, r="value", theta="Dataset", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [5.9,1], "dtick": 1}})
fig_spider_by_dataset

In [ ]:
import plotly.graph_objects as go
import scikit_posthocs as sp

def conover_test(df):
    df = df.unstack(level=1).rank().T.reset_index().melt(id_vars=['level_0', 'Dataset'], var_name='Method', value_name='Rank')
    posthoc = sp.posthoc_conover(df, group_col='Method', val_col='Rank', p_adjust='bonferroni')

    return posthoc

def pretty_conover(df):
    posthoc = conover_test(df)

    p_threshold = 0.05 / len(posthoc.columns)
    print(p_threshold)

    def highlight_below_threshold(val):
        color = 'background-color: turquoise' if val < p_threshold else ''
        return color

    return posthoc.style.applymap(highlight_below_threshold).format("{:.2e}")

def plot_on_number_line(s, title, axis_label, connections=[], textpositions=["top center","top center","top center","top center","top center"], range_max=None, range_min=None):
    colors = [color for _, color in zip(methods, px.colors.qualitative.Plotly)]

    fig = go.Figure(go.Scatter(
        x=s.values,
        y=[0]*len(s),
        mode="markers+text",
        text=s.index,
        textfont=dict(color=colors),
        textposition=textpositions,
        marker=dict(size=12, color=colors)
    ))

    if not range_max:
        range_max = s.max()
    if not range_min:
        range_min = s.min()
    h_margin = 0.1*(range_max-range_min)

    for i, (m1, m2) in enumerate(connections):
        y_offset = (i+1)*0.07
        gray_shade = 50 + 200 * (i/len(connections))
        line_color = f"rgb({gray_shade},{gray_shade},{gray_shade})"
        fig.add_shape(
            type="line",
            x0=s[m1], x1=s[m2],
            y0=y_offset, y1=y_offset,
            line=dict(color=line_color, width=2),
            layer="below"
        )

    # Thin horizontal number line
    fig.add_shape(type="line",
                x0=range_min, x1=range_max,
                y0=0, y1=0,
                line=dict(color="black", width=1),
                layer="below")
    
    # Start line
    fig.add_shape(type="line",
                x0=range_min, x1=range_min,
                y0=-1, y1=1,
                line=dict(color="white", width=1),
                layer="below")

    # Layout tweaks for minimal look
    fig.update_yaxes(visible=False)
    fig.update_xaxes(range=[range_min-h_margin, range_max+h_margin], showgrid=True, zeroline=False)
    fig.update_layout(
        height=200,
        xaxis_title=axis_label,
        yaxis_title="",
        showlegend=False,
        margin=dict(t=60, b=60, l=150, r=150),
        title=title,
    )

    fig.update_layout(width=1000, height=210)
    fig.show()
    fig.update_layout(
        title=None,
        margin=dict(t=60, b=60, l=50, r=50)
    )
    fig.write_image(f"{title}.pdf")
    return fig


def plot_average_rankings(df, title, connections=[], textpositions=["top center","bottom center","top center","top center","top center"]):
    s = rank(df).mean(axis=1)

    range_max = len(df.index.get_level_values(level="Method").unique())

    return plot_on_number_line(s, title=title, axis_label="Average Rankings", connections=connections, textpositions=textpositions, range_min=1, range_max=range_max)

def rank(df):
    return df.unstack(level=1).rank()



In [789]:
just_ranks = df_all.unstack(level=1).rank().transpose().reset_index(drop=True)
import plotly.express as px

rank_histogram = px.histogram(just_ranks.astype(int), nbins=6, labels={'value':'rank'})
rank_histogram

In [790]:
connections = [("TimeGAN", "Time-Transformer"), ("Time-Transformer", "TimeVQVAE")] # according to connover test.
all_measures_ranking = plot_average_rankings(df_no_copy, connections=connections, title="All Measures")
all_measures_ranking

In [791]:
pretty_conover(df_no_copy)

0.01


,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,2.88e-23,1.27e-15,1.09e-27,3.01e-41
Time-Transformer,2.88e-23,1.00e+00,1.97e-01,1.00e+00,1.22e-05
TimeGAN,1.27e-15,1.97e-01,1.00e+00,3.63e-03,2.69e-11
TimeVQVAE,1.09e-27,1.00e+00,3.63e-03,1.00e+00,2.65e-03
TransFusion,3.01e-41,1.22e-05,2.69e-11,2.65e-03,1.00e+00


In [792]:
connections=[("TimeVQVAE", "TimeGAN")]
plot_average_rankings(
    df_all
        .drop(index="JustCopy", level="Method")
        .drop(columns=["PS", "Time"]), 
    title="Fidelity",
    textpositions=["top center","bottom center","top left","top center","top center"],
    connections=connections)

In [793]:
pretty_conover(df_no_copy.drop(columns=["PS", "Time"]))

0.01


,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,1.33e-22,3.73e-20,3.07e-30,3.29e-43
Time-Transformer,1.33e-22,1.00e+00,1.00e+00,2.03e-01,2.17e-08
TimeGAN,3.73e-20,1.00e+00,1.00e+00,2.10e-02,2.64e-10
TimeVQVAE,3.07e-30,2.03e-01,2.10e-02,1.00e+00,1.30e-03
TransFusion,3.29e-43,2.17e-08,2.64e-10,1.30e-03,1.00e+00


In [794]:
connections=[("TTS-GAN", "TimeGAN"), ("TimeVQVAE", "TransFusion")]
plot_average_rankings(
    df_no_copy["PS"].to_frame(), 
    connections=connections, 
    title="Utility", 
    textpositions=["top center","top right","bottom right","top center","top center"]
)

In [795]:
pretty_conover(df_no_copy["PS"].to_frame())

0.01


,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
TTS-GAN,1.00e+00,1.00e+00,7.85e-01,1.00e+00,7.75e-04
Time-Transformer,1.00e+00,1.00e+00,1.00e+00,1.00e+00,2.23e-02
TimeGAN,7.85e-01,1.00e+00,1.00e+00,1.00e+00,7.99e-02
TimeVQVAE,1.00e+00,1.00e+00,1.00e+00,1.00e+00,2.23e-02
TransFusion,7.75e-04,2.23e-02,7.99e-02,2.23e-02,1.00e+00


In [796]:
from scipy.stats import friedmanchisquare

df = df_no_copy.unstack(level=1).T
ranks = df.rank(axis=0, method='average', ascending=False)

stat, p = friedmanchisquare(*[ranks.loc[m].values for m in ranks.index])
p

8.69339661590917e-22

In [797]:
import pandas as pd

def leaderboard(df):
    out = pd.DataFrame(index=df.index.levels[1], columns=df.columns)

    for lvl2 in df.index.levels[1]:
        subset = df.xs(lvl2, level=1)
        min_idx = subset.idxmin()
        out.loc[lvl2] = min_idx


    def color_cells(value):
        if value == "TTS-GAN":
            return 'background-color: yellow'
        if value == "TransFusion":
            return 'background-color: lightgreen'
        if value == "TimeGAN":
            return 'background-color: lightblue'
        if value == "Time-Transformer":
            return 'background-color: pink'
        if value == "TimeVQVAE":
            return 'background-color: turquoise'
        if value == "JustCopy":
            return 'background-color: lightseagreen'

    return out.style.applymap(color_cells)

def worst_board(df):
    out = pd.DataFrame(index=df.index.levels[1], columns=df.columns)

    for lvl2 in df.index.levels[1]:
        subset = df.xs(lvl2, level=1)
        max_idx = subset.idxmax()
        out.loc[lvl2] = max_idx


    def color_cells(value):
        if value == "TTS-GAN":
            return 'background-color: yellow'
        if value == "TransFusion":
            return 'background-color: lightgreen'
        if value == "TimeGAN":
            return 'background-color: lightblue'
        if value == "Time-Transformer":
            return 'background-color: pink'
        if value == "TimeVQVAE":
            return 'background-color: turquoise'
        if value == "JustCopy":
            return 'background-color: lightseagreen'

    return out.style.applymap(color_cells)


leaderboard(df_all)


,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D3,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D4,TransFusion,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D5,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D6,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D7,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy


In [798]:
leaderboard(df_no_copy)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer
D3,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D4,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer,Time-Transformer,Time-Transformer
D5,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,Time-Transformer,Time-Transformer,Time-Transformer
D6,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D7,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer


In [799]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["TTS-GAN", "TimeGAN"])])

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D3,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D4,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D5,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D6,TTS-GAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D7,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN


In [800]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["Time-Transformer", "TimeGAN"])])

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer
D3,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D4,Time-Transformer,TimeGAN,Time-Transformer,Time-Transformer,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D5,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D6,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D7,Time-Transformer,TimeGAN,Time-Transformer,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer


In [801]:
worst_board(df_all)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN
D3,Time-Transformer,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,Time-Transformer,TTS-GAN,TTS-GAN,TimeGAN
D4,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TimeGAN
D5,TimeVQVAE,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN
D6,TimeGAN,Time-Transformer,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TransFusion
D7,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TimeGAN


In [802]:
rankings

PS                            DS                 ...  DTW  \
Dataset            D2   D3   D4   D5   D6   D7   D2   D3   D4   D5  ...   D4   
Method                                                              ...        
TransFusion       2.0  1.0  1.0  1.0  1.0  1.0  2.0  1.0  1.0  1.0  ...  3.0   
TimeVQVAE         3.0  3.0  2.0  5.0  4.0  3.0  3.0  3.0  4.0  3.0  ...  2.0   
Time-Transformer  4.0  5.0  3.0  4.0  2.0  2.0  4.0  4.0  3.0  4.0  ...  1.0   
TimeGAN           1.0  2.0  4.0  2.0  5.0  4.0  1.0  2.0  2.0  2.0  ...  4.0   
TTS-GAN           5.0  4.0  5.0  3.0  3.0  5.0  5.0  5.0  5.0  5.0  ...  5.0   

                                Time                           
Dataset            D5   D6   D7   D2   D3   D4   D5   D6   D7  
Method                                                         
TransFusion       2.0  3.0  3.0  2.0  3.0  3.0  3.0  5.0  4.0  
TimeVQVAE         3.0  1.0  1.0  3.0  2.0  2.0  2.0  2.0  2.0  
Time-Transformer  1.0  2.0  2.0  1.0  1.0  1.0  1.0  1.0  1.0  
TimeGAN           4.0  4.0  4.0  4.0  5.0  5.0  4.0  4.0  5.0  
TTS-GAN           5.0  5.0  5.0  5.0  4.0  4.0  5.0  3.0  3.0  

[5 rows x 60 columns]

In [803]:
df = df_no_copy.unstack(level=1).rank()
utility_cols = [col for col in df.columns if col[0] in ["PS"]]

fidelity_cols = [col for col in df.columns if col[0] not in ["PS", "Time"]]

timing_cols = [col for col in df.columns if col[0] in ["Time"]]

avg_utility = df[utility_cols].mean(axis=1)
avg_fidelity = df[fidelity_cols].mean(axis=1)
avg_timing = df[timing_cols].mean(axis=1)

new_df = pd.DataFrame({
    "Utility": avg_utility,
    "Fidelity": avg_fidelity,
    "Time": avg_timing
})

new_df.style.background_gradient(cmap=cm)

,Utility,Fidelity,Time
Method,,,
TransFusion,1.166667,1.708333,3.333333
TimeVQVAE,3.333333,2.437500,2.166667
Time-Transformer,3.333333,2.875000,1.000000
TimeGAN,3.000000,3.020833,4.500000
TTS-GAN,4.166667,4.958333,4.000000


In [815]:
df = df_no_copy.unstack(level=1).rank()
model_based_cols = [col for col in df.columns if col[0] in ["PS", "DS"]]

feature_based_cols = [col for col in df.columns if col[0] not in ["PS", "Time", "DS", "DTW", "ED"]]

efficiency_based_cols = [col for col in df.columns if col[0] in ["Time"]]

distance_based_cols = [col for col in df.columns if col[0] in ["DTW", "ED"]]

avg_model_based = df[model_based_cols].mean(axis=1)
avg_feature_based = df[feature_based_cols].mean(axis=1)
avg_efficiency_based = df[efficiency_based_cols].mean(axis=1)
avg_distance_based = df[distance_based_cols].mean(axis=1)

rankings_tsgbench_metric_groups = pd.DataFrame({
    "Model Based": avg_model_based,
    "Feature Based": avg_feature_based,
    "Efficiency Based": avg_efficiency_based,
    "Distance Based": avg_distance_based
})

rankings_tsgbench_metric_groups.style.background_gradient(cmap=cm)

,Model Based,Feature Based,Efficiency Based,Distance Based
Method,,,,
TransFusion,1.166667,1.400000,3.333333,2.750000
TimeVQVAE,3.166667,2.633333,2.166667,1.666667
Time-Transformer,3.583333,3.033333,1.000000,2.000000
TimeGAN,2.583333,2.966667,4.500000,3.583333
TTS-GAN,4.500000,4.966667,4.000000,5.000000


In [829]:
plot_on_number_line(rankings_tsgbench_metric_groups["Model Based"], "Model Based Measures Ranking", "Average Rank")

In [804]:
timings.set_index("Method").drop(index="JustCopy")

,D2,D3,D4,D5,D6,D7
Method,,,,,,
TimeGAN,259,1373,1195,304,1252,1385
Time-Transformer,13,16,28,66,77,55
TransFusion,43,222,454,201,1387,856
TTS-GAN,646,660,649,649,660,645
TimeVQVAE,73,94,94,74,88,86


In [805]:
average_timings = timings.set_index("Method").mean(axis=1).to_frame(name="Average Wall Clock Time").drop(index="JustCopy")


plot_on_number_line(\
    average_timings["Average Wall Clock Time"], \
    title = "Average Wall Clock Time", \
    axis_label = "minutes", \
    range_min=1, \
    range_max=1000, \
    textpositions=["top center","bottom center","top center","bottom center","top center"])

# Visualizations for paper

## Benchmark Results

In [806]:
leaderboard(df_all)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D3,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D4,TransFusion,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D5,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D6,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D7,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy


The overall leaderboard shows the best scoring method on each dataset in each measure. As is clearly visible, the simple "JustCopy" method, that emulates the behavioiur a perfectly overfitted model, scores best in all but one cases.

conclusion notes:
- This shows, that the benchmark does not have a metric to measure a key aspect of a good generator: novelty. (related work: quality of a generator: novelty)

In [807]:
leaderboard(df_no_copy)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer
D3,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D4,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer,Time-Transformer,Time-Transformer
D5,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,Time-Transformer,Time-Transformer,Time-Transformer
D6,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D7,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer


The leaderboard without the cheating JustCopy method TransFusion is the best method in most pairings, but all methods except for TTS-GAN are best in some measure-dataset combination. Notably TransFusion is not best in ED and DTW in most datasets, overtaken by TimeVQVAE and Time-Transformer respectively. One of the clearest results is Time-Transformer being consistently the fastest method. TTS-GAN makes no appearence on the leaderboard.

In [808]:
worst_board(df_all)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN
D3,Time-Transformer,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,Time-Transformer,TTS-GAN,TTS-GAN,TimeGAN
D4,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TimeGAN
D5,TimeVQVAE,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN
D6,TimeGAN,Time-Transformer,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TransFusion
D7,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TTS-GAN,TimeGAN


Looking at the worst scores TTS-GAN is very consistently ranks last in all metrics on all datasets, except for PS and Time, where it only scores last on 3 and 2 of 6 datasets respectively.

In [809]:
style_results_table(
    df_all
        .groupby(level="Method").mean()
        .loc[ranking_order]
    )

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Method,,,,,,,,,,
JustCopy,0.113,0.012,-6.73×10⁻¹³,0.0231,0,0,0,0,0,0
TransFusion,0.114,0.115,2.41×10⁻²,0.344,0.122,0.176,0.718,2.1,8.13,527
TimeVQVAE,0.146,0.379,2.3,0.509,0.551,0.315,1.27,1.99,7.73,84.8
Time-Transformer,0.143,0.396,2.11,0.534,0.495,0.378,1.41,2.03,7.91,42.5
TimeGAN,0.143,0.286,2.49,0.583,0.717,0.346,1.31,2.14,8.22,961
TTS-GAN,0.279,0.476,5.25×10¹⁶,1.05,6.71,1,2.16,4.68×10⁷,1.52×10⁸,652


For a better understanding of the margins by which the methods differ, we can look at the mean scores of each methods fidelity metrics over all datasets. The table paints a pretty clear picture in two regards. First, JustCopy plays the benchmark, scoring perfect or near perfect scores in all measures. Second, TTS-GANs scores are worse than all other benchmarked methods by, in cases, orders of magnitude in all metrics except for Time.

conclusion notes:
- TTS-GAN off the chart bad
- Just Copy perfect scores (whats going on with MDD and C-FID?)
- ED and DTW show almost no difference between methods (except for TTS-GAN)

In [810]:
all_measures_ranking

To arrive at an overall Ranking of the benchmarked methods, I followed the approach presented by [todo cite angeetal] to compute the average ranking of each method in each measure across all datasets. To statistically validate the rankings, a Friedman test [cite] along with a Conover's test for ranking comparisons was employed. Figure [above] presents the average rankings of each method. Methods that the Conover's test did not find statistically different are connected with a horizontal bar.

TransFusion ranks best, followed by TimeVQVAE, Time-Transformer, TimeGAN in the middle, and last by a margin TTS-GAN. The Conover's test clearly separates TransFusion to rank highest and TTS-GAN to rank lowest. In the middle the Conover's test only separates TimeVQVAE from TimeGAN, with Time-Transformer between them not ranking significantly lower than one or higher than the other.

In [811]:
fig_spider_by_dataset.show()
fig_spider_by_metric

The spider plot shows the average ranking of each method per metric. The average ranking on the metrics measuring the closeness of the distributions (KD, SD, ACD, MDD and C-FID) is consistent with the overall ranking result, TransFusion being clearly first, TTS-GAN clearly last and Time-Transformer, TimeGAN and TimeVQVAE close together in the middle, with TimeVQVAE consistently taking the second rank. The model based metrics (PS, DS) show the same first and last place, but interestingly TimeGAN is consistently second. The average ranking for the distance based measures (ED, DTW) differs strongly from the overall ranking, with TimeVQVAE and Time-Transformer ranking first and second, followed by TransFusion in just the third place, TimeGAN fourth and TTS-GAN last. The average ranking of the methods in the Time metric is again distinctly from the overall ranking, with Time-Transformer consistently ranking first, followed by Time-VQVAE, TransFusion, TTS-GAN and last, TimeGAN.

conclusion notes:
- overall ranking biased towards the thingn measured by the most measures
- distance based measures have higher values on longer sequence lengths
- ED and DTW rankings look more dramatic than the numeric results would warrant

### <s>Impact of Training Data Sequence Lenght on Benchmark Performance</s>

## Observed Benchmark Issues

In [812]:
leaderboard(df_all)

,PS,DS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D3,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D4,TransFusion,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D5,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D6,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
D7,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy,JustCopy
